# Time Series

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 1000)

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

import os, sys
path_add = os.path.abspath(os.pardir)
if path_add not in sys.path:
    sys.path.append(path_add)

## Agenda

SWBAT:

- Manipulate datetime objects;
- Use rolling and weighted averages to inspect time series;
- Conduct Dickey-Fuller Tests for stationarity.

## Intro

The study of time series has arisen because certain sorts of data streams are heavily dependent on the flow of time. Of course, we have not totally ignored time as a feature up to this point. The selling price of a house probably *does* have some relation to the season or the year as real estate markets grow and decline with certain temporally-indexed economic changes etc. But surely time is not the most important predictor of house price. Square footage would likely be more strongly correlated with price than would date of sale.

But there are other sorts of data that more readily lend themselves to a temporal analysis. One canonical example is numbers from a stock exchange: First, data from stock tickers often arrive as numbers anchored to consecutive units of time. I get the selling price for some stock on January 1, say, and the next bit of information I gain will be the selling price for that stock on January 2. (We'll explore this feature of time series below.) Second, and more important, if I'm interested in actually *predicting* the selling price of a stock for, say, tomorrow, then very likely one piece of very salient (i.e. *correlated*) information would be the selling price of that stock *today*.

Let's look at some time series data plots.

## Visualizing Trends

In [ ]:
# Define a function that will help us load and
# clean up a dataset.

def load_trend(trend_name='football', country_code='us'):
    df = pd.read_csv('../data/google-trends_'
                     + trend_name + '_'
                     + country_code
                     + '.csv').iloc[1:, :]
    df.columns = ['counts']
    df['counts'] = df['counts'].str.replace('<1', '0').astype(int)
    return df

In [ ]:
df = load_trend(**{'trend_name': 'data-science', 'country_code': 'us'})


In [ ]:
trends = [
    {'trend_name': 'data-science', 'country_code': 'us'},
    {'trend_name': 'football', 'country_code': 'us'},
    {'trend_name': 'football', 'country_code': 'uk'},
    {'trend_name': 'coronavirus', 'country_code': 'us'},
    {'trend_name': 'trump', 'country_code': 'us'},
    {'trend_name': 'taxes', 'country_code': 'us'},
    {'trend_name': 'avengers', 'country_code': 'us'}
]

In [ ]:
np.random.shuffle(trends)

In [ ]:
trend_dfs = [load_trend(**trend) for trend in trends]

In [ ]:
# Let's see if we can guess which is which just by looking
# at their graphs.

fig, axs = plt.subplots(len(trend_dfs), 1, figsize=(8, 10))
plt.tight_layout()
for i, trend_df in enumerate(trend_dfs):
    ax = axs[i]
    #ax.set_title(str(trends[i]))
    ax.plot(np.array(trend_df.index), trend_df['counts'])
    ticks = ax.get_xticks()
    ax.set_ylim((0, 100))
    ax.set_xticks([tick for tick in ticks if tick%24 == 0])

## Series as Both Predictor and Target?

Often, the phenomenon we want to capture with a time series is a dataset being correlated with *itself*.

Well, of course every dataset is perfectly correlated with itself. But what we're after now is the idea that a series is correlated with *earlier versions* of itself.

Consider the problem of trying to predict tomorrow's closing price for some stock on the market. One may consider lots of features, like what sort of company it is to which the stock belongs or whether that company has been in the news recently.

But it is very often the case that one of the most helpful predictors of tomorrow's price is *today's* price. And so we want to build a model where one of our predictors is an earlier version of our target.

## EDA

Let's import some data on **gun violence in Chicago**.

[source](https://data.cityofchicago.org/Public-Safety/Gun-Crimes-Heat-Map/iinq-m3rg)

In [ ]:
ts = pd.read_csv('../data/Gun_Crimes_Heat_Map.csv')

In [ ]:
ts.head()

Let's look at some summary stats:

In [ ]:
print(f"There are {ts.shape[0]} records in our timeseries")

In [ ]:
# Definitely some messy input of our Desciption data
ts['Description'].value_counts()

In [ ]:
height = ts['Description'].value_counts()[:10]
offense_names = ts['Description'].value_counts()[:10].index

fig, ax = plt.subplots()
sns.barplot(height, offense_names, color='r', ax=ax)
ax.set_title('Mostly Handgun offenses');

In [ ]:
# Mostly non-domestic offenses

fig, ax = plt.subplots()
sns.barplot( ts['Domestic'].value_counts().index, 
             ts['Domestic'].value_counts(),  
             palette=[ 'r', 'b'], ax=ax
           )

ax.set_title("Overwhelmingly Non-Domestic Offenses");

In [ ]:
# Mostly non-domestic offenses

arrest_rate = ts['Arrest'].value_counts()[1]/len(ts)

fig, ax = plt.subplots()

sns.barplot( ts['Arrest'].value_counts().index, 
             ts['Arrest'].value_counts(), 
             palette=['r', 'g'], ax=ax
           )

ax.set_title(f'{arrest_rate: 0.2%} of Total Cases\n Result in Arrest');

In [ ]:
fig, ax = plt.subplots()
sns.barplot( ts['Year'].value_counts().index, 
             ts['Year'].value_counts(),  
             color= 'r', ax=ax
           )

ax.set_title("Offenses By Year");

While this does show some interesting information that will be relevant to our time series analysis, we are going to get more granular.

## Datetime Objects

Datetime objects make our time series modeling lives easier.  They will allow us to perform essential data prep tasks with a few lines of code.  

We need our timeseries **index** to be datetime objects, since our models will rely on being able to identify the previous chronological value.

There is a `datetime` [library](https://docs.python.org/2/library/datetime.html), and inside `pandas` there is a datetime module as well as a to_datetime() function.

For time series modeling, the first step often is to make sure that the index is a datetime object.

There are a few ways to **reindex** our series to datetime. 

We can use `pandas.to_datetime()` method:

In [ ]:
ts.index

In [ ]:
ts.set_index(pd.to_datetime(ts['Date']), drop=True, inplace=True)

Or, we can parse the dates directly on import:

In [ ]:
ts = pd.read_csv('../data/Gun_Crimes_Heat_Map.csv', index_col='Date', parse_dates=True)

In [ ]:
print(f"Now our index is a {type(ts.index)}")

In [ ]:
ts.head()

Datetime objects include aspects of the date as attributes, like month and year:

In [ ]:
ts.index[0]

In [ ]:
ts.index[0].month

In [ ]:
ts.index[0].year

We can easily see now whether offenses happen, for example, during business hours.

In [ ]:
fig, ax = plt.subplots()

ts['hour'] = ts.index
ts['hour'] = ts.hour.apply(lambda x: x.hour)
ts['business_hours'] = ts.hour.apply(lambda x: 9 <= x <= 17)

bh_ratio = ts.business_hours.value_counts()[1]/len(ts)

x = ts.business_hours.value_counts().index
y = ts.business_hours.value_counts()
sns.barplot(x=x, y=y)

ax.set_title(f'{bh_ratio: 0.2%} of Offenses\n Happen Btwn 9 and 5');

## Upsampling and Downsampling

With a Datetime index, we also have new abilities, such as **resampling**.

To create our timeseries, we will count the number of gun offenses reported per day.

In [ ]:
ts.resample('D')

There are many possible units for resampling, each with its own alias:

<table style="display: inline-block">
    <caption style="text-align: center"><strong>TIME SERIES OFFSET ALIASES</strong></caption>
<tr><th>ALIAS</th><th>DESCRIPTION</th></tr>
<tr><td>B</td><td>business day frequency</td></tr>
<tr><td>C</td><td>custom business day frequency (experimental)</td></tr>
<tr><td>D</td><td>calendar day frequency</td></tr>
<tr><td>W</td><td>weekly frequency</td></tr>
<tr><td>M</td><td>month end frequency</td></tr>
<tr><td>SM</td><td>semi-month end frequency (15th and end of month)</td></tr>
<tr><td>BM</td><td>business month end frequency</td></tr>
<tr><td>CBM</td><td>custom business month end frequency</td></tr>
<tr><td>MS</td><td>month start frequency</td></tr>
<tr><td>SMS</td><td>semi-month start frequency (1st and 15th)</td></tr>
<tr><td>BMS</td><td>business month start frequency</td></tr>
<tr><td>CBMS</td><td>custom business month start frequency</td></tr>
<tr><td>Q</td><td>quarter end frequency</td></tr>
<tr><td></td><td><font color=white>intentionally left blank</font></td></tr></table>

<table style="display: inline-block; margin-left: 40px">
<caption style="text-align: center"></caption>
<tr><th>ALIAS</th><th>DESCRIPTION</th></tr>
<tr><td>BQ</td><td>business quarter endfrequency</td></tr>
<tr><td>QS</td><td>quarter start frequency</td></tr>
<tr><td>BQS</td><td>business quarter start frequency</td></tr>
<tr><td>A</td><td>year end frequency</td></tr>
<tr><td>BA</td><td>business year end frequency</td></tr>
<tr><td>AS</td><td>year start frequency</td></tr>
<tr><td>BAS</td><td>business year start frequency</td></tr>
<tr><td>BH</td><td>business hour frequency</td></tr>
<tr><td>H</td><td>hourly frequency</td></tr>
<tr><td>T, min</td><td>minutely frequency</td></tr>
<tr><td>S</td><td>secondly frequency</td></tr>
<tr><td>L, ms</td><td>milliseconds</td></tr>
<tr><td>U, us</td><td>microseconds</td></tr>
<tr><td>N</td><td>nanoseconds</td></tr></table>

When resampling, we have to provide a rule to resample by, and an **aggregate function**.

**To upsample** is to increase the frequency of the data of interest.  
**To downsample** is to decrease the frequency of the data of interest.

For our purposes, we will downsample, and  count the number of occurences per day.

In [ ]:
ts.resample('D').count()

Our time series will consist of a series of counts of gun reports per day.

In [ ]:
# ID is unimportant. We could choose any column, since the counts are the same.
ts = ts.resample('D').count()['ID']

In [ ]:
ts

Let's visualize our timeseries with a plot.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(ts.index, ts.values)
ax.set_title('Gun Crimes per day in Chicago')
ax.set_ylabel('Reported Gun Crimes');

There seems to be some abnormal activity happening towards the end of our series.

**[sun-times](https://chicago.suntimes.com/crime/2020/6/8/21281998/chicago-deadliest-day-violence-murder-history-police-crime)**

In [ ]:
ts.sort_values(ascending=False)[:10]

Let's treat the span of days from 5-31 to 6-03 as outliers. 

There are several ways to do this, but let's first remove the outliers, and populate an an empty array with the original date range. That will introduce us to the `pandas.date_range()` method.

In [ ]:
daily_count = ts[ts < 90]
ts_dr = pd.date_range(daily_count.index[0], daily_count.index[-1])
ts_daily = np.empty(shape=len(ts_dr))
ts_daily = pd.Series(ts_daily)
ts_daily = ts_daily.reindex(ts_dr)
ts = ts_daily.fillna(daily_count)

In [ ]:
ts

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ts.plot(ax=ax)
ax.set_title('Gun Crimes in Chicago with Deadliest Days Removed');

Let's zoom in on that week again:

In [ ]:
fig, ax = plt.subplots()
ax.plot(ts[(ts.index > '2020-05-20') 
                 & (ts.index < '2020-06-07')]
       )
ax.tick_params(rotation=45)
ax.set_title('We have some gaps now');

The datetime object allows us several options of how to fill those gaps:

In [ ]:
# .ffill()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 5))
ax1.plot(ts.ffill()[(ts.index > '2020-05-20') 
                 & (ts.index < '2020-06-07')]
       )
ax1.tick_params(rotation=45)
ax1.set_title('Forward Fill')

ax2.plot(ts[(ts.index > '2020-05-20') 
                 & (ts.index < '2020-06-07')]
       )
ax2.tick_params(rotation=45)
ax2.set_title('Original');

In [ ]:
# .bfill()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 5))
ax1.plot(ts.bfill()[(ts.index > '2020-05-20') 
                 & (ts.index < '2020-06-07')]
       )
ax1.tick_params(rotation=45)
ax1.set_title('Back Fill')

ax2.plot(ts[(ts.index > '2020-05-20') 
                 & (ts.index < '2020-06-07')]
       )
ax2.tick_params(rotation=45)
ax2.set_title('Original');

In [ ]:
# .interpolate()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 5))
ax1.plot(ts.interpolate()[(ts.index > '2020-05-20') 
                 & (ts.index < '2020-06-07')]
       )
ax1.tick_params(rotation=45)
ax1.set_title('Interpolation')

ax2.plot(ts[(ts.index > '2020-05-20') 
                 & (ts.index < '2020-06-07')]
       )
ax2.tick_params(rotation=45)
ax2.set_title('Original');

Let's proceed with the interpolated data.

In [ ]:
ts = ts.interpolate()
ts.isna().sum()

Now that we've cleaned up a few data points, let's downsample to the week level.  

In [ ]:
ts_weekly = ts.resample('W').mean()

In [ ]:
ts_weekly.plot();

## Visual Diagnostics with Moving Averages

Let's begin considering some models for our data.

These are not useful for prediction just yet, but they will lead us towards our prediction models.

### Simple Moving Average

A simple moving average consists of an average across a specified window of time. 

The datetime index allows us to calculate simple moving averages via the `rolling()` function.

The rolling function calculates a statistic across a moving **window**, which we can change with the window parameter.

In [ ]:
ts_weekly.rolling(window=4)

Let's calculate a month long moving average

In [ ]:
ts_weekly.rolling(4).mean()[:10]

This is simply the avarage of a datapoint and the previous three data points:

In [ ]:
ts_weekly[:4]

In [ ]:
ts_weekly[:4].mean() 

In [ ]:
ts_weekly[:4].mean() == ts_weekly.rolling(4).mean()[3]

In [ ]:
sma_week = ts_weekly.rolling(4).mean()

fig, ax = plt.subplots(figsize=(10, 5))

ts_weekly.plot(ax=ax, label='raw')
sma_week.plot(ax=ax, label='rolling average')

plt.legend();

As we can see from the plot below, the simple moving average **smooths** out the series. Smoothing can help visualize the underlying pattern. It can also be a very simple predictive model, where we just project the mean out into the future.

In [ ]:
# Let's zoom in

fig, ax = plt.subplots(figsize=(10,5))

ts_weekly[-100:].plot(ax=ax, c='r', label='raw')
sma_week[-100:].plot(ax=ax, c='b', label='rolling average')

plt.legend();

The simple moving avereage tracks fairly well, but does not reach to the peaks and valleys of the original distribution.

If we plot the moving average across 52 weeeks, we can see a smooth trend across a year.

In [ ]:
sma_week = ts_weekly.rolling(52).mean()

fig, ax = plt.subplots(figsize=(10, 5))

ts_weekly.plot(ax=ax, label='raw')
sma_week.plot(ax=ax, label='rolling average')

plt.legend();

### Exponentially Weighted Moving Average 

An alternative to SMA is the [EWMA](https://en.wikipedia.org/wiki/Moving_average#Exponential_moving_average). The exponentially weighted average gives more weight to the points closer to the date in question.  With EWMA, the average will track more closely to the peaks and valleys. If there are extreme historical values in the dataset, the EWMA will be less skewed than the SMA.

In [ ]:
ts_ex_ewm = ts_weekly.ewm(alpha=0.5).mean()[:10]
ts_ex_ewm

The higher the $\alpha$ parameter, the closer the EWMA will be to the actual value of the point.

In [ ]:
ts_ex_ewm = ts_weekly.ewm(alpha=0.99).mean()[:10]
ts_ex_ewm

In [ ]:
ts_weekly[:10]

Let's plot our rolling statistics with some different windows:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ts_weekly[-100:].plot(ax=ax, c='r', label='Original')
ts_weekly.rolling(4).mean().dropna()[-100:].plot(ax=ax, c='g', label='SMA')
ts_weekly.ewm(span=4).mean().dropna()[-100:].plot(ax=ax, c='b', label='EWMA')

plt.legend();

Again, if we zoom in to the year level, we can see peaks and valleys according to the seasons.  

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ts_weekly.plot(ax=ax, c='r', label='Original')
ts_weekly.rolling(52).mean().dropna().plot(ax=ax, c='g', label='SMA')
ts_weekly.ewm(span=52).mean().dropna().plot(ax=ax, c='b', label='EWMA')

plt.legend();

We can also plot rolling averages for the variance and standard deviation.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ts_weekly.plot(ax=ax, c='r', label='Original')
ts_weekly.rolling(4).var().dropna().plot(ax=ax, c='g', label='SMA')
ts_weekly.ewm(span=4).var().dropna().plot(ax=ax, c='b', label='EWMA')

plt.legend();

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ts_weekly.rolling(52).var().dropna().plot(ax=ax, c='g', label='SMA')
ts_weekly.ewm(span=52).var().dropna().plot(ax=ax, c='b', label='EWMA')

plt.legend();

If we zoom in on our standard deviation, we can see that the variance of our data has quite a fluctuation at different moments in time.  When we are building our models, we will want to remove this variability, or our models will have different performance at different times.

## Components of Time Series Data

A time series in general is supposed to be affected by four main components, which can be separated from the observed data. These components are: *Trend, Cyclical, Seasonal and Irregular* components.

- **Trend** : The long term movement of a time series. For example, series relating to population growth, number of houses in a city etc. show upward trend.
- **Seasonality** : Fluctuation in the data set that follow a regular pattern due to outside influences. For example sales of ice-cream increase in summer, or daily web traffic.
- **Cyclical** : When data exhibit rises and falls that are not of fixed period.  Think of business cycles which usually last several years, but where the length of the current cycle is unknown beforehand.
- **Irregular**: Are caused by unpredictable influences, which are not regular and also do not repeat in a particular pattern. These variations are caused by incidences such as war, strike, earthquake, flood, revolution, etc. There is no defined statistical technique for measuring random fluctuations in a time series.


*Note: Many people confuse cyclic behaviour with seasonal behaviour, but they are really quite different. If the fluctuations are not of fixed period then they are cyclic; if the period is unchanging and associated with some aspect of the calendar, then the pattern is seasonal.*

The statsmodels seasonal decompose can also help show us the trends in our data.

In [ ]:
decomposition = seasonal_decompose(ts_weekly)
fig = plt.figure()
fig = decomposition.plot()
fig.set_size_inches(15, 8)

## Statistical stationarity: 

When building our models, we will want to account for these trends somehow.  Time series whose mean and variance have trends across time will be difficult to predict out into the future. 

A **stationary time series** is one whose statistical properties such as mean, variance, autocorrelation, etc. are all constant over time. Most statistical forecasting methods are based on the assumption that the time series can be rendered approximately stationary (i.e., "stationarized") through the use of mathematical transformations. A stationarized series is relatively easy to predict: you simply predict that its statistical properties will be the same in the future as they have been in the past!

It may seem counterintuitive that, for modeling purposes, we want our time series not to be a function of time! But the basic idea is the familiar one that we want our datapoints to be mutually *independent*. For more on this topic, see [here](https://stats.stackexchange.com/questions/19715/why-does-a-time-series-have-to-be-stationary).

One way of testing for stationarity is to use the Dickey-Fuller Test. The statsmodels version returns the test statistic and a p-value, relative to the null hypothesis that the series in question is NOT stationary. For more, see [this Wikipedia page](https://en.wikipedia.org/wiki/Augmented_Dickey%E2%80%93Fuller_test).


<h3 style="text-align: center;">Constant Mean</p>



<img src='../img/mean_nonstationary.webp'/>

<h3 style="text-align: center;">Constant Variance</p>


<img src='../img/variance_nonstationary.webp'/>

<h3 style="text-align: center;">Constant Covariance</p>


<img src='../img/covariance_nonstationary.webp'/>

While we can get a sense of how stationary our data is with visuals, the Dickey-Fuller test gives us a quantitatitive measure.

Here the null hypothesis is that the TS is non-stationary. If the test statistic is less than the critical value, we can reject the null hypothesis and say that the series is stationary.

In [ ]:
#create a function that will help us to quickly test stationarity
def test_stationarity(timeseries, window):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(window=window).mean()
    rolstd = timeseries.rolling(window=window).std()

    #Plot rolling statistics:
    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries.iloc[window:], color='blue', label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic',
                                             'p-value', '#Lags Used',
                                             'Number of Observations Used'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)' %key] = value
    print (dfoutput)

In [ ]:
test_stationarity(ts_weekly, 52)

The time series does not pass the test of stationarity for $\alpha=0.05$.

### How to stationarize time series data

A series of steps can be taken to stationarize your data - also known -  as removing trends (linear trends, seasonaility/periodicity, etc - more details on transformations <a href='http://people.duke.edu/~rnau/whatuse.htm'>here</a>).


One way to remove trends is to difference our data.  
Differencing is performed by subtracting the previous observation (lag=1) from the current observation.

In [ ]:
ts_weekly.diff().dropna()[:5]

In [ ]:
ts_weekly.diff().dropna().plot();

In [ ]:
ts_weekly[:5]

Sometimes, we have to difference the differenced data (known as a second difference) to achieve stationary data. <b>The number of times we have to difference our data is the order of differencing</b> - we will use this information when building our model.

In [ ]:
#Second order difference:

ts_weekly.diff().diff().dropna()[:5]

In [ ]:
# We can also apply seasonal differences:
    
ts_weekly.diff(52).dropna()[:10]

Let's difference our data and see if it improves Dickey-Fuller Test

In [ ]:
ts_weekly.diff().dropna()

In [ ]:
test_stationarity(ts_weekly.diff().dropna(), 52)

Much better!